#### Install client library for the Google API

In [1]:
!pip install --upgrade google-api-python-client

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 60 kB 811 kB/s eta 0:00:011
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.9.3
    Uninstalling google-api-python-client-1.9.3:
      Successfully uninstalled google-api-python-client-1.9.3


#### Import required libraries and objects
* The <b>requests</b> module will be required for us to make our own REST API calls
* The <b>google.oauth2.servie_account</b> object allows us to generate credentials from a service account file
* The <b>googleapiclient.discovery</b> allows us to connected to available GCP services

In [2]:
import requests

from google.oauth2 import service_account
import googleapiclient.discovery

ModuleNotFoundError: No module named 'requests'

#### Set the details required to generate credentials
Possible scopes to use with the billing API:
* https://www.googleapis.com/auth/cloud-platform
* https://www.googleapis.com/auth/cloud-billing
* https://www.googleapis.com/auth/cloud-billing.readonly

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/cloud-billing']

SERVICE_ACCOUNT_FILE = 'service-account-file.json'

#### Generate credentials

In [ ]:
credentials = service_account\
              .Credentials\
              .from_service_account_file(SERVICE_ACCOUNT_FILE, 
                                         scopes = SCOPES)
        
credentials

#### Connect to the billing account service

In [ ]:
billing_service = googleapiclient\
                  .discovery\
                  .build('cloudbilling', 
                         'v1', 
                         credentials = credentials)

billing_service

In [ ]:
billing_service.billingAccounts()

#### The list() function lists the billing accounts associated with the user
This returns an HttpRequest object which needs to be executed to send the request (and get a response from) the billing service

In [ ]:
billing_service.billingAccounts()\
               .list()

In [ ]:
billing_service.billingAccounts()\
               .list()\
               .execute()

#### Get the projects linked to a billing account

In [ ]:
projects = billing_service.billingAccounts()\
                          .projects()\
                          .list(name = 'billingAccounts/010F91-8460C4-934F45')\
                          .execute()

projects

#### Get the billing info for a project

In [ ]:
proj_billing = billing_service\
               .projects()\
               .getBillingInfo(name = 'projects/spikey-dev')\
               .execute()
            
proj_billing

#### Examine the access token in the credentials object
We will need to use this in a REST API call

In [ ]:
credentials.token

#### Generate the headers for a REST call
Which includes the access token

In [ ]:
headers = {'Authorization': 'Bearer %s' % credentials.token }
headers

#### Make a GET request
Get the list of billing accounts for the user - similar to what we did previously with the client library

In [ ]:
response = requests.get('https://cloudbilling.googleapis.com/v1/billingAccounts', 
                        headers = headers)

print(response.text)

#### Get the projects linked to a billing account
Also done previously with the client library

In [ ]:
url = "https://cloudbilling.googleapis.com/v1/billingAccounts/010F91-8460C4-934F45/projects"

url

In [ ]:
response = requests.get(url,
                        headers = headers)

print(response.text)

#### Create a POST request to change the billing account for a project
Use the json library to create the body for the POST request

In [ ]:
import json

In [ ]:
body_dict = {
              "name": "projects/spikey-dev/billingInfo",
              "projectId": "spikey-dev",
              "billingAccountName": "billingAccounts/01EB73-BD73AE-23D5A1",
              "billingEnabled": False
            }

In [ ]:
body_json = json.dumps(body_dict)

body_json

In [ ]:
url = "https://cloudbilling.googleapis.com/v1/projects/spikey-dev/billingInfo"
    
response = requests.put(url, 
                        headers = headers, 
                        data = body_json)

print(response.text)

In [ ]:
url = "https://cloudbilling.googleapis.com/v1/billingAccounts/010F91-8460C4-934F45/projects"
    
response = requests.get(url, 
                        headers = headers)

print(response.text)

In [ ]:
response = requests.get('https://cloudbilling.googleapis.com/v1/services', 
                        headers = headers)

print(response.text)

In [ ]:
response = requests.get('https://cloudbilling.googleapis.com/v1/services/04C4-B046-D8B2/skus', 
                        headers = headers)

print(response.text)